In [ ]:
import pandas as pd
import numpy as np
import holidays
import os
import random

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from pycaret.regression import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(22) # Seed 고정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# [AutoGluon : 시계열 접근]
- https://dacon.io/competitions/official/236176/codeshare/9188?page=1&dtype=recent 참고

In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.0/224.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.3/372.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 88.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 9.1 MB/s e

- 모듈 로드 및 전처리

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

data = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/data/train.csv', encoding = 'utf-8')

In [ ]:
train_df = data[['ID','timestamp','supply(kg)', 'price(원/kg)']]
train_df['item_id'] = train_df.ID.str[0:6]

- 학습

In [ ]:
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231119_212604/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']


- 전체 데이터 대상으로 학습

In [ ]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	2.89    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 2.95 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

- 예측

In [ ]:
pred = predictor.predict(data, random_seed=42, )

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [ ]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3239.581356  1115.400218  1871.337201  2387.234516   
        2023-03-05   640.292341 -1906.907531 -1033.140405  -405.292988   
        2023-03-06  3086.232107  -222.601819   937.860125  1741.971521   
        2023-03-07  3474.040741  -306.212860  1026.834329  1916.485574   
        2023-03-08  3301.059558  -795.400467   672.816662  1654.980400   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   512.885946  -140.463583    98.579706   259.877887   
        2023-03-28   531.818537  -172.643246   105.021120   270.000988   
        2023-03-29   533.859562  -172.104770    95.930224   269.589513   
        2023-03-30   504.552140  -220.374391    57.276367   234.925213   
        2023-03-31   500.475837  -258.004951    41.872213   217.233330   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2846.946821  3256.888358  3673.723258  4114.187794   
        2023-03-05   120.668804   603.887325  1107.159191  1638.389031   
        2023-03-06  2424.111275  3069.929694  3700.993037  4392.985153   
        2023-03-07  2727.344801  3429.908480  4163.527830  4975.725891   
        2023-03-08  2505.153305  3285.946871  4125.951959  4986.719692   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   393.197674   516.780239   640.509019   772.273519   
        2023-03-28   409.329377   535.974160   664.457750   802.358695   
        2023-03-29   409.834497   539.737691   668.533382   808.932731   
        2023-03-30   379.806693   514.442756   646.499132   790.885202   
        2023-03-31   365.366881   501.947502   644.219771   801.594832   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4626.787590  5385.592817  
        2023-03-05  2258.316397  3120.524909  
        2023-03-06  5224.030541  6334.069277  
        2023-03-07  5910.007268  7256.537739  
        2023-03-08  6030.813054  7434.261870  
...                         ...          ...  
RD_F_J  2023-03-27   930.480937  1157.824100  
        2023-03-28   965.153112  1192.610080  
        2023-03-29   973.826490  1203.970543  
        2023-03-30   959.454167  1193.224369  
        2023-03-31   973.713454  1210.885058  

[1092 rows x 10 columns]

In [ ]:
 pred = pred.reset_index()
 pred

,item_id,timestamp,mean,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
0,TG_A_J,2023-03-04,3239.581356,1115.400218,1871.337201,2387.234516,2846.946821,3256.888358,3673.723258,4114.187794,4626.787590,5385.592817
1,TG_A_J,2023-03-05,640.292341,-1906.907531,-1033.140405,-405.292988,120.668804,603.887325,1107.159191,1638.389031,2258.316397,3120.524909
2,TG_A_J,2023-03-06,3086.232107,-222.601819,937.860125,1741.971521,2424.111275,3069.929694,3700.993037,4392.985153,5224.030541,6334.069277
3,TG_A_J,2023-03-07,3474.040741,-306.212860,1026.834329,1916.485574,2727.344801,3429.908480,4163.527830,4975.725891,5910.007268,7256.537739
4,TG_A_J,2023-03-08,3301.059558,-795.400467,672.816662,1654.980400,2505.153305,3285.946871,4125.951959,4986.719692,6030.813054,7434.261870
...,...,...,...,...,...,...,...,...,...,...,...,...
1087,RD_F_J,2023-03-27,512.885946,-140.463583,98.579706,259.877887,393.197674,516.780239,640.509019,772.273519,930.480937,1157.824100
1088,RD_F_J,2023-03-28,531.818537,-172.643246,105.021120,270.000988,409.329377,535.974160,664.457750,802.358695,965.153112,1192.610080
1089,RD_F_J,2023-03-29,533.859562,-172.104770,95.930224,269.589513,409.834497,539.737691,668.533382,808.932731,973.826490,1203.970543
1090,RD_F_J,2023-03-30,504.552140,-220.374391,57.276367,234.925213,379.806693,514.442756,646.499132,790.885202,959.454167,1193.224369


- 예측 결과 후처리

In [ ]:
# 일요일은 0으로 처리
pred['timestamp'] = pd.to_datetime(pred['timestamp'])
pred['day_of_week'] = pred['timestamp'].dt.dayofweek  # 월요일(0)부터 일요일(6)까지
pred.loc[pred['day_of_week'] == 6, 'mean'] = 0
pred = pred.drop(columns=['day_of_week'])

In [ ]:
# mean 열이 0보다 작은 경우, item_id와 day_of_week별로 평균값으로 대체

# timestamp 열을 날짜로 변환
pred['timestamp'] = pd.to_datetime(pred['timestamp'])

# day_of_week 열을 추가하여 해당 날짜의 요일을 저장
pred['day_of_week'] = pred['timestamp'].dt.dayofweek  # 월요일(0)부터 일요일(6)까지

mask = pred['mean'] < 0
pred.loc[mask, 'mean'] = pred.groupby(['item_id', 'day_of_week'])['mean'].transform('mean')

In [ ]:
sum(pred['mean'] < 0)

0

In [ ]:
sub['answer_AG'] = pred.reset_index()['mean']

In [ ]:
sub.to_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_try_only_AG.csv', index=False)

# [최종 앙상블]

- 최종 submission 체크

In [ ]:
final_ag_sub = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_try_only_AG.csv', encoding = 'utf-8')
final_ml_sub = pd.read_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_try_only_ML.csv', encoding = 'utf-8')

In [ ]:
sub

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0
...,...,...
1087,RD_F_J_20230327,0
1088,RD_F_J_20230328,0
1089,RD_F_J_20230329,0
1090,RD_F_J_20230330,0


- 성능이 좋았던 AutoGluon 모델에 가중치를 두고, AutoML 학습 결과의 특성도 반영한다.

In [ ]:
sub_1 = sub.copy()
sub_1['answer'] = ((2 * final_ag_sub['answer_AG']) + final_ml_sub['answer_ML']) / 3

In [ ]:
#sub_1.drop(['answer_ML', 'answer_AG'], axis = 1, inplace = True)
sub_1

,ID,answer
0,TG_A_J_20230304,3065.740948
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3070.822313
3,TG_A_J_20230307,3279.285979
4,TG_A_J_20230308,3163.419058
...,...,...
1087,RD_F_J_20230327,490.073700
1088,RD_F_J_20230328,500.061851
1089,RD_F_J_20230329,504.522234
1090,RD_F_J_20230330,489.266402


In [ ]:
sub_1.to_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_ens_1.csv', index=False)

In [ ]:
sub_2 = sub.copy()
sub_2['answer'] = ((3 * final_ag_sub['answer_AG']) + final_ml_sub['answer_ML']) / 4
sub_2

,ID,answer
0,TG_A_J_20230304,3109.201050
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3074.674762
3,TG_A_J_20230307,3327.974670
4,TG_A_J_20230308,3197.829183
...,...,...
1087,RD_F_J_20230327,495.776762
1088,RD_F_J_20230328,508.001022
1089,RD_F_J_20230329,511.856566
1090,RD_F_J_20230330,493.087837


In [ ]:
sub_2.to_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_ens_2.csv', index=False)

In [ ]:
sub_3 = sub.copy()
sub_3['answer'] = ((4 * final_ag_sub['answer_AG']) + final_ml_sub['answer_ML']) / 5
sub_3

,ID,answer
0,TG_A_J_20230304,3135.277111
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3076.986231
3,TG_A_J_20230307,3357.187884
4,TG_A_J_20230308,3218.475258
...,...,...
1087,RD_F_J_20230327,499.198598
1088,RD_F_J_20230328,512.764525
1089,RD_F_J_20230329,516.257165
1090,RD_F_J_20230330,495.380697


In [ ]:
sub_3.to_csv('/content/drive/MyDrive/Dacon/23_제주특산물가격예측/submissions/final_ens_3.csv', index=False)